# Data extraction via MT5 Api

Each trading symbol has its own process for the wine mt5 api. This is so that in production, we're able to reduce latency (at the cost of higher memory consumption) by pulling from one or more brokers in parallel.

### IMPORTANT - start mt5 apis

Before running this notebooks, first launch the mt5 apis from the docker terminal using the command: 

`releat launch-all-mt5-apis`

Alternatively, you can run it from your local terminal and execute on your docker container, replace `<container-name>` with the name of the container, which should either be `releat` or `releat-dc` depending on how you set it up:

`docker exec -it <container-name> releat launch-all-mt5-apis`

Launching wine processes from jupyter notebooks is unstable and mostly fails to start

In [ ]:
from releat.utils.configs.constants import mt5_api_port_map
from concurrent.futures import ThreadPoolExecutor
from releat.workflows.service_manager import kill_processes, get_pids, stop_mt5
from releat.utils.logging import get_logger
from releat.data.extractor import download_tick_data
import logging
from datetime import datetime
logger = get_logger(__name__, log_level=logging.INFO)

In [ ]:
# start date - datetime must be in this format
dt0 = datetime.strptime("2023-09-06 10:00:00.000","%Y-%m-%d %H:%M:%S.%f")
# end date - excludes boundary
dt1 = datetime.strptime("2023-09-06 10:01:01.001","%Y-%m-%d %H:%M:%S.%f")
# data_mode - either demo or live
data_mode = "demo"
# check api - whether to check the connection, if failed, it will try to initialize the
# connection. checking is skipped at inference
check_api = False

dl_args = []

for broker, port_map in mt5_api_port_map.items():
    for symbol, port in port_map.items():
        # general is the port used for other interactions with mt5, i.e. order and
        # getting position
        if symbol!='general':
            dl_arg = [broker,symbol,dt0,dt1,data_mode,check_api]
            dl_args.append(dl_arg)

print("An example of input list to the download_tick_data function:")  
print(dl_args[0])

In [ ]:
# show example output of function
download_tick_data(*dl_args[1])

In [ ]:
pool = ThreadPoolExecutor(len(dl_args))

### Compare data extraction speed

When run in parallel, the time taken to download data is at least 2x as fast as compared to in sequence. Note results will vary depending on internet speed and whether results are cached

In [ ]:
%%timeit
for dl_arg in dl_args:
    download_tick_data(*dl_arg)

In [ ]:
%%timeit
list(pool.map(lambda p: download_tick_data(*p), dl_args))

### Clean processes by deleting MT5 instances and apis

In [ ]:
# kill mt5
stop_mt5()

In [ ]:
# kill mt5 api process ids
pids = get_pids("wineserver")
kill_processes(pids)
print(f"mt5 apis stopped - process ids {pids} killed")

# kill wine processes
pids = get_pids("python.exe")
kill_processes(pids)